In [ ]:
import matplotlib.pyplot as plt
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import numpy as np
import pandas as pd
import scipy as sp
from scipy import interpolate
import os
import urllib.request
import tempfile

gdoc.setNoScroll()

## Creation of the data base for thickness

In [ ]:
# Use either Oise_Thickness.csv, Oise_Thickness_withSides.csv or Oise_Ztop500.csv
# Data files are available here: https://soft.mines-paristech.fr/gstlearn/data-latest/Alluvial
filename = gdoc.loadData("Alluvial", "Oise_Ztop500.csv")
csv = gl.CSVformat(True, 0, ";", ",", "9999")
data = gl.Db.createFromCSV(filename, csv, False)
data

In [ ]:
data.setLocator("X",gl.ELoc.X,0)
data.setLocator("Y",gl.ELoc.X,1)
data.setLocator("Z",gl.ELoc.L,0)
data.setLocator("Z_1",gl.ELoc.U,0)
sf = gl.DbStringFormat()
sf.setFlags(flag_stats=True) 
thickness = data.getWithinBounds(0)
err = data.addColumns(thickness,"Elev",gl.ELoc.Z)
err = gl.DbHelper.db_duplicate(data)
data

In [ ]:
res = plt.scatter(data.getColumn("X"), data.getColumn("Y"), c=data.getColumn("Elev"))

In [ ]:
res = plt.hist(thickness)

In [ ]:
#data.dumpToNF("Oise_Data_Elev500.ascii")

## Creation of the grid

### 1) Creation of the polygon

In [ ]:
# Data files are available here: https://soft.mines-paristech.fr/gstlearn/data-latest/Alluvial
filename = gdoc.loadData("Alluvial", "Oise_Shapefile_AlluvialPlain.csv")
poly = gl.Polygons.createFromCSV(filename, csv, False)
ax = gp.plot(poly)

In [ ]:
grid = gl.DbGrid.create(nx = [3300,400],dx = [50.,50.], x0 = [630000.,6865000.],angles=[40,0])

In [ ]:
fig, ax = gp.init()
ax.raster(grid,"x1",alpha=0.3)
ax.symbol(data)
gp.close()

In [ ]:
err = gl.db_polygon(grid,poly)
grid

### 2) Creation of angles

In [ ]:
# Data files are available here: https://soft.mines-paristech.fr/gstlearn/data-latest/Alluvial
filename = gdoc.loadData("Alluvial", "Oise_Shapefile_Centerline.csv")
## Loading Centerline data from file
df1 = pd.read_csv (filename, sep=';')
xc = list(df1['X'])
yc = list(df1['Y'])
x1=xc[1:499]
y1=yc[1:499]

# Data files are available here: https://soft.mines-paristech.fr/gstlearn/data-latest/Alluvial
filename = gdoc.loadData("Alluvial", "Oise_Shapefile_AlluvialPlain.csv")
## Loading alluvial plain contours from file & Separate into two polylines 
df = pd.read_csv (filename, sep=';')
xp = list(df['X'])
yp = list(df['Y'])
x2=xp[1:xp.index(max(xp))]
x3=xp[len(xp):xp.index(max(xp))+1:-1]
y2=yp[1:xp.index(max(xp))]
y3=yp[len(xp):xp.index(max(xp))+1:-1]

## Adding supplementary control points at the edges
# coordinates of extremes
XA1=640000
YA1=6875000
XB1=740000
YB1=6955000

XA2=630000
YA2=6890000
XB2=735000
YB2=6980000

# Forming two supplementary vectors at the edges
n=19
x4= np.zeros(n)
y4= np.zeros(n)
x5= np.zeros(n)
y5= np.zeros(n)
for i in range(0,len(x4)):
    x4[i]=(XA1+(XB1-XA1)/(n-1)*i)
    y4[i]=(YA1+(YB1-YA1)/(n-1)*i)
    x5[i]=(XA2+(XB2-XA2)/(n-1)*i)
    y5[i]=(YA2+(YB2-YA2)/(n-1)*i)

In [ ]:
# Gradient method
#  https://stackoverflow.com/questions/28269379/curve-curvature-in-numpy
dx_dt1 = np.gradient(x1)
dy_dt1 = np.gradient(y1)
velocity = np.array([ [dx_dt1[i], dy_dt1[i]] for i in range(dx_dt1.size)])
ds_dt1 = np.sqrt(dx_dt1 * dx_dt1 + dy_dt1 * dy_dt1)
tangent1 = np.array([1/ds_dt1] * 2).transpose() * velocity

dx_dt2 = np.gradient(x2)
dy_dt2 = np.gradient(y2)
velocity2 = np.array([ [dx_dt2[i], dy_dt2[i]] for i in range(dx_dt2.size)])
ds_dt2 = np.sqrt(dx_dt2 * dx_dt2 + dy_dt2 * dy_dt2)
tangent2 = np.array([1/ds_dt2] * 2).transpose() * velocity2

dx_dt3 = np.gradient(x3)
dy_dt3 = np.gradient(y3)
velocity3 = np.array([ [dx_dt3[i], dy_dt3[i]] for i in range(dx_dt3.size)])
ds_dt3 = np.sqrt(dx_dt3 * dx_dt3 + dy_dt3 * dy_dt3)
tangent3 = np.array([1/ds_dt3] * 2).transpose() * velocity3

dx_dt4 = np.gradient(x4)
dy_dt4 = np.gradient(y4)
velocity4 = np.array([ [dx_dt4[i], dy_dt4[i]] for i in range(dx_dt4.size)])
ds_dt4 = np.sqrt(dx_dt4 * dx_dt4 + dy_dt4 * dy_dt4)
tangent4 = np.array([1/ds_dt4] * 2).transpose() * velocity4

dx_dt5 = np.gradient(x5)
dy_dt5 = np.gradient(y5)
velocity5 = np.array([ [dx_dt5[i], dy_dt5[i]] for i in range(dx_dt5.size)])
ds_dt5 = np.sqrt(dx_dt5 * dx_dt5 + dy_dt5 * dy_dt5)
tangent5 = np.array([1/ds_dt5] * 2).transpose() * velocity5
    
tangent=np.concatenate((tangent1,tangent2,tangent3,tangent4,tangent5),axis=0) 

x0=np.concatenate((x1,x2,x3,x4,x5),axis=0) 
y0=np.concatenate((y1,y2,y3,y4,y5),axis=0) 
u0 = tangent[:, 0]
v0 = tangent[:, 1]

In [ ]:
# Numpy arctan2 Method
angles = np.arctan2(v0, u0) * 180 / np.pi
res = plt.hist(angles)

In [ ]:
# Interpolating the angles into angles map -> https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.griddata.html
xx = np.linspace(630000, 742000, 2240)
yy = np.linspace(6875000, 6982200, 2144)
xx, yy = np.meshgrid(xx, yy)

angles.shape=(np.size(angles),)
np.shape(angles)

points = np.transpose(np.vstack((x0, y0)))
angles_interp = interpolate.griddata(points, angles, (xx, yy), method='linear')

xx1 = np.concatenate(xx)
yy1 = np.concatenate(yy)
angles_interp1 = np.concatenate(angles_interp)

In [ ]:
#Create a point Db point using xx, yy, angles_interp
dbv = gl.Db()
dbv.addColumns(xx1,"xx",gl.ELoc.X, 0)
dbv.addColumns(yy1,"yy",gl.ELoc.X, 1)
dbv.addColumns(angles_interp1,"angles_interp",gl.ELoc.Z, 0)
dbv.display()
grid.setLocator("Pol*")
grid.display()

In [ ]:
#Tester la valeur retournée par migrate :
grid.deleteColumn("Migr*")
gl.migrateMulti(dbv, grid, ["angles_interp"])
grid.setLocator("Migrate*", gl.ELoc.Z)
grid.display()

In [ ]:
grid.useSel = False
grid.setLocator("Polygon")
uid_selection=grid.addSelectionByLimit("*angles_interp*", gl.Limits([-10], [10]), "angle_define")

In [ ]:
xplt = grid.getColumnByLocator(gl.ELoc.X,0,useSel=True)
yplt = grid.getColumnByLocator(gl.ELoc.X,1,useSel=True)
aplt = grid.getColumnByLocator(gl.ELoc.Z,0,useSel=True)

### 3) Creation of a selection (polygon + borders by morphological dilation)

In [ ]:
grid.setLocator("Poly*", gl.ELoc.SEL)

In [ ]:
vmin = 0.5
vmax = 1.5
nxy = grid.getNXs()
image2 = gl.BImage(nxy)

tab = grid.getColumn("Polygon",useSel=False)
image = gl.morpho_double2image(nxy,tab,vmin,vmax)
localVD = gl.VectorDouble(len(tab))
gl.morpho_dilation(0, [3,3], image, image2)
for i in range(10):
    gl.morpho_dilation(0, [1,1], image, image2)
    gl.morpho_dilation(0, [1,1], image2, image)
#gl.morpho_dilation(0, [1,1], image, image2)

gl.morpho_image2double(image2, 0, 1., 0., localVD)

In [ ]:
grid["res"]=localVD.getVector()

In [ ]:
grid.setLocator("res",gl.ELoc.SEL)
grid

In [ ]:
res = gp.raster(grid,"Migrate.angles_interp",cmap="turbo", flagLegend = True)

### 4) Save the result

In [ ]:
#grid.dumpToNF("Oise_GridAnglesModifFinal.ascii")

# Start Kriging here once data is properly saved

In [ ]:
#grid = gl.DbGrid.createFromNF("Oise_GridAnglesModifFinal.ascii")
grid

In [ ]:
# Modify the ranges to 8000, 800 if wished
ranges = [2000,200]
#If old method is set to True, a classical SPDE kriging will be used
oldMethod = False

In [ ]:
# Load data, use either with or without the sides; For the top of the alluvial plain use "Oise_Data_Elev500.ascii"
# Data files are available here: https://soft.mines-paristech.fr/gstlearn/data-latest/Alluvial
filename = gdoc.loadData("Alluvial", "Oise_Data_ThicknessSides.ascii")
data = gl.Db.createFromNF(filename)
data

In [ ]:
# Selection to calculate variogram only on non-zero values
data["selVario"]=(data["Duplicate"]*(data["ThicknessSides"]>0))
data.setLocators(["selVario"],gl.ELoc.SEL)
data

In [ ]:
myVarioParamBidir = gl.VarioParam()
mydir = gl.DirParam(40,800.,0.5,45.,0,0,np.nan,np.nan,0.,[],[1,1])
myVarioParamBidir.addDir(mydir)
mydir = gl.DirParam(20,400.,0.5,45.,0,0,np.nan,np.nan,0.,[],[-1,1])
myVarioParamBidir.addDir(mydir)
myVarioBidir = gl.Vario(myVarioParamBidir)
err = myVarioBidir.compute(data,gl.ECalcVario.VARIOGRAM)
myVarioBidir.display()
ax = gp.varmod(myVarioBidir,idir=0)
ax.decoration(title="Bi-directional Variogram for thickness(45°)")
ax = gp.varmod(myVarioBidir,idir=1)
ax.decoration(title="Bi-directional Variogram for thickness(45/135°)")

In [ ]:
model = gl.Model.createFromDb(data)

structs = [gl.ECov.NUGGET,gl.ECov.MATERN]
cons1P = gl.ConsItem.define(gl.EConsElem.PARAM,1, type = gl.EConsType.EQUAL,value = 1)

a = gl.Constraints()
a.addItem(cons1P)

err = model.fit(myVarioBidir,structs,constraints=a)
model.display()
ax = gp.varmod(myVarioBidir,model,idir=0)
ax = gp.varmod(myVarioBidir,model,idir=1)
plt.show()

#model.setMean(100)
model.display()

In [ ]:
data.setLocators(["Duplicate"],gl.ELoc.SEL)

In [ ]:
if not oldMethod :
    model = gl.Model.createFromDb(data)
    covMatern = gl.CovAniso.createAnisotropic(type = gl.ECov.MATERN,ranges=ranges,sill=4.7,param=1,ctxt=model.getContext())
    covNugget = gl.CovAniso.createIsotropic(type = gl.ECov.NUGGET,sill=2.2,ctxt=model.getContext(),range=1)
    model.addCov(covNugget)
    model.addCov(covMatern)

In [ ]:
grid.setLocator("Polygon.*", gl.ELoc.SEL)

In [ ]:
model.setDriftIRF(order = 0)
model

In [ ]:
if not oldMethod:
    model.getCovAniso(1).makeAngleNoStatDb("Migrate.angles_interp",0,grid)

In [ ]:
data.setLocators(["ThicknessNosides"],gl.ELoc.Z,cleanSameLocator=True)

In [ ]:
# Basic SPDE simulation 
spde = gl.SPDE(model,grid,data,gl.ESPDECalcMode.KRIGING)
uid_result= spde.compute(grid)

In [ ]:
#Plot kriging
fig, ax = gp.init(figsize=(20,12))
gp.raster(grid,"spde.ThicknessSides.estim",cmap="turbo", flagLegend = True)
gp.close()

# Save the current grid, which can be loaded again at the start of kriging (cell #26)

In [ ]:
# modify the grid according to your wishes
#grid["Old_Variable"]=grid["New_variable"]
#grid.deleteColumn("TheVariableYouWant")
#grid["BedrockElevation"]=grid["PlainElevation"]-grid["AlluviumThickness"]

In [ ]:
#Save the grid
#grid.dumpToNF("Oise_KrigingResults_Thickness.ascii")

# Uncertainties estimation based on standard deviation

In [ ]:
# Calculation of n simulations to estimate standard deviation (best is to change to nbsimu=100)
error=gl.simulateSPDE(data,grid,model,nbsimu=10)

In [ ]:
# Calculation of Stdv by hand: gl.EStatOption.STDV in the future
grid["MeanSimu"]=grid["SimuSPDE.Thickness*"].mean(1)
grid["StdevSimu"]=np.zeros(grid["SimuSPDE.Thickness*"].shape[0])
nc=grid["SimuSPDE.Thickness*"].shape[1]
for i in range(nc):
    grid["StdevSimu"]+=((grid["SimuSPDE.Thickness*"][:,i]-grid["MeanSimu"])**2)/nc
grid["StdevSimu"]=grid["StdevSimu"]**0.5